In [13]:
import graphviz


In [34]:
file1 = open('input.txt', 'r')
lines = file1.readlines()

In [44]:
dict(filter(lambda x: "con" in x[1]["value"], instructions.items()))

{'a': {'type': '%', 'value': ['inv', 'con'], 'state': 'off'},
 'b': {'type': '%', 'value': ['con'], 'state': 'off'}}

In [55]:
instructions = {}
for line in lines:
    instruction = line.strip().split(' -> ')
    if instruction[0] == "broadcaster":
        instructions[instruction[0]] = {"type": "broadcaster", "value": instruction[1].split(", ")}
    elif instruction[0][0] == "%":
        instructions[instruction[0][1:]] = {"type": instruction[0][0], "value": instruction[1].split(", "), "state": "off"}#flipflop
    elif instruction[0][0] == "&":
        instructions[instruction[0][1:]] = {"type": instruction[0][0], "value": instruction[1].split(", "), "last": {}}#Conjunction

for key in instructions:
    if instructions[key]["type"] == "&":
        for i in list(dict(filter(lambda x: key in x[1]["value"], instructions.items())).keys()):
            instructions[key]["last"][i] = "low"

def process_button(current_command, source, signal):
    print(f"{source} -> {signal} -> {current_command}")
    counters = (0,0)
    change = (0,1) if signal == "low" else (1,0)
    counters = tuple(map(sum, zip(counters, change)))

    if not current_command in instructions:
        return counters

    if instructions[current_command]["type"] == "broadcaster":
        for i in instructions[current_command]["value"]:
            counters = tuple(map(sum, zip(counters, process_button(i, current_command, signal))))
        return counters

    if instructions[current_command]["type"] == "&":#Conjunction
        instructions[current_command]["last"][source] = signal
        if all(list(map(lambda x: x == "high", instructions[current_command]["last"].values()))):
            for i in instructions[current_command]["value"]:
                counters = tuple(map(sum, zip(counters,process_button(i, current_command, "low"))))
        else:
            for i in instructions[current_command]["value"]:
                counters = tuple(map(sum, zip(counters,process_button(i, current_command, "high"))))
        return counters

    if instructions[current_command]["type"] == "%": #flipflop
        if signal == "low":
            if instructions[current_command]["state"] == "off":
                instructions[current_command]["state"] = "on"
                for i in instructions[current_command]["value"]:
                    counters = tuple(map(sum, zip(counters, process_button(i, current_command, "high"))))
            else:
                instructions[current_command]["state"] = "off"
                for i in instructions[current_command]["value"]:
                    counters = tuple(map(sum, zip(counters, process_button(i, current_command, "low"))))
        return counters


process_button("broadcaster", "button", "low")



button -> low -> broadcaster
broadcaster -> low -> a
a -> high -> inv
inv -> low -> b
b -> high -> con
con -> high -> output
a -> high -> con
con -> low -> output


(4, 4)

In [31]:
counters = (0, 0)
for i in range(1000):
    counters = tuple(map(sum, zip(counters, process_button("broadcaster", "button", "low"))))
print(f"part1: {counters[0]*counters[1]}")

True

In [26]:
dot = graphviz.Digraph(comment='The Machine')
for key in instructions:
    dot.node(key, key)
for key in instructions:
    for i in instructions[key]["value"]:
        dot.edge(key, i)

dot.render('round-table.gv', view=True)

'round-table.gv.pdf'